### Bước 1: Đọc dữ liệu từ file `Combined_Results.xlsx`


In [12]:
import pandas as pd

# Đọc file Combined_Results.xlsx
df = pd.read_excel('Combined_Results.xlsx', engine='openpyxl')

# Hiển thị thông tin cơ bản về dữ liệu
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 70 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Mã Sinh Viên                        426 non-null    object
 1   Bảo Trì Hệ Thống                    426 non-null    object
 2   Đại Số Tuyến Tính                   426 non-null    object
 3   Nhập Môn Ngành Cntt                 426 non-null    object
 4   Tin Học Đại Cương                   426 non-null    object
 5   Vật Lý Điện Từ                      426 non-null    object
 6   Kỹ Năng Mềm                         426 non-null    object
 7   Triết Học Mác- Lênin                426 non-null    object
 8   Giải Tích 1                         426 non-null    object
 9   Kỹ Thuật Lập Trình                  426 non-null    object
 10  Lập Trình Nâng Cao                  426 non-null    object
 11  Kinh Tế Chính Trị Mác- Lênin        426 non-null    object



### Bước 2: Tách dữ liệu thành tập huấn luyện và tập kiểm tra


In [13]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Đọc file Combined_Results.xlsx
df = pd.read_excel('Combined_Results.xlsx', engine='openpyxl')

# Giả sử cột 'Xếp loại' là cột mục tiêu
X = df.drop(columns=['Xếp loại', 'Mã Sinh Viên'])
y = df['Xếp loại']

# Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape)

(298, 68) (128, 68)


Preprocessing

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Xác định các cột phân loại và số
categorical_cols = X_train.select_dtypes(include=['object']).columns
numerical_cols = X_train.select_dtypes(include=['number']).columns

# Bộ tiền xử lý cho dữ liệu số
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Điền giá trị thiếu bằng trung bình
    ('scaler', StandardScaler())  # Chuẩn hóa dữ liệu
])

# Bộ tiền xử lý cho dữ liệu phân loại
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Điền giá trị thiếu bằng giá trị phổ biến nhất
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # Mã hóa OneHot với đầu ra dạng dense
])

# Kết hợp các bộ tiền xử lý
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

# Định nghĩa các mô hình học máy
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Naive Bayes': GaussianNB()
}

# Lưu trữ kết quả
results = {}

# Huấn luyện và đánh giá từng mô hình
for name, model in models.items():
    # Tạo pipeline cho từng mô hình
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    # Huấn luyện mô hình
    pipeline.fit(X_train, y_train)
    
    # Dự đoán và tính độ chính xác
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    
    # # In ma trận nhầm lẫn
    # cm = confusion_matrix(y_test, y_pred)
    # print(f"Ma trận nhầm lẫn cho {name}:")
    # sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    # plt.title(f"Confusion Matrix for {name}")
    # plt.xlabel("Predicted")
    # plt.ylabel("Actual")
    # plt.show()

# Tìm mô hình có độ chính xác cao nhất
# In bảng so sánh hiệu suất
results_df = pd.DataFrame(list(results.items()), columns=['Thuật toán', 'Accuracy test'])
results_df['Accuracy train'] = [
    accuracy_score(y_train, pipeline.fit(X_train, y_train).predict(X_train)) for pipeline in [
        Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)]) for model in models.values()
    ]
]
results_df['Chênh lệch'] = results_df['Accuracy train'] - results_df['Accuracy test']
results_df = results_df.sort_values(by='Chênh lệch', ascending=False)

print("Bảng kiểm tra overfitting của các thuật toán:")
print(results_df)

# Tìm mô hình có Accuracy cao nhất
best_model = max(results, key=results.get)
print(f"Thuật toán có Accuracy cao nhất: {best_model} với Accuracy test: {results[best_model]:.2f}")

Bảng kiểm tra overfitting của các thuật toán:
            Thuật toán  Accuracy test  Accuracy train  Chênh lệch
3        Decision Tree       0.773438        1.000000    0.226562
2  K-Nearest Neighbors       0.656250        0.852349    0.196099
1                  SVM       0.835938        0.966443    0.130505
0        Random Forest       0.875000        1.000000    0.125000
4          Naive Bayes       0.757812        0.865772    0.107959
Thuật toán có Accuracy cao nhất: Random Forest với Accuracy test: 0.88


In [16]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

# Kiểm tra và xử lý giá trị NaN trong dữ liệu
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Định nghĩa các mô hình và siêu tham số
model_params = {
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__max_depth': [None, 10, 20],
        }
    },
    'SVM': {
        'model': SVC(),
        'params': {
            'classifier__C': [0.1, 1, 10],
            'classifier__kernel': ['linear', 'rbf']
        }
    },
    'K-Nearest Neighbors': {
        'model': KNeighborsClassifier(),
        'params': {
            'classifier__n_neighbors': [3, 5, 7],
            'classifier__weights': ['uniform', 'distance']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {
            'classifier__max_depth': [None, 10, 20],
            'classifier__min_samples_split': [2, 5, 10]
        }
    },
    'Naive Bayes': {
        'model': GaussianNB(),
        'params': {}  # Không có siêu tham số
    }
}

# Số lượng lần chia tập dữ liệu trong cross-validation
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)  # Giảm n_splits để tránh cảnh báo

# Lưu trữ kết quả
results = []

# Huấn luyện và đánh giá từng mô hình
for name, config in model_params.items():
    model = config['model']
    params = config['params']
    
    # Tạo pipeline cho từng mô hình
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    # Sử dụng GridSearchCV để điều chỉnh siêu tham số
    grid_search = GridSearchCV(
        pipeline, 
        param_grid=params, 
        cv=cv, 
        scoring='accuracy', 
        n_jobs=-1, 
        error_score=np.nan
    )
    grid_search.fit(X_train, y_train)
    
    # Lấy mô hình tốt nhất
    best_model = grid_search.best_estimator_
    
    # Dự đoán trên tập huấn luyện và tập kiểm tra
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)
    
    # Tính độ chính xác
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    # Lưu kết quả
    results.append({
        'Thuật toán': name,
        'Accuracy train': train_accuracy,
        'Accuracy test': test_accuracy,
    })

# Tạo DataFrame từ kết quả
results_df = pd.DataFrame(results)

# Tính chênh lệch giữa độ chính xác trên tập huấn luyện và tập kiểm tra
results_df['Chênh lệch'] = results_df['Accuracy train'] - results_df['Accuracy test']

# Sắp xếp bảng theo chênh lệch giảm dần
results_df = results_df.sort_values(by='Chênh lệch', ascending=False)

# In bảng so sánh
print("Bảng kiểm tra overfitting của các thuật toán:")
print(results_df)

# Tìm mô hình có Accuracy cao nhất
best_result = max(results, key=lambda x: x['Accuracy test'])
print(f"Thuật toán có Accuracy cao nhất: {best_result['Thuật toán']} với Accuracy test: {best_result['Accuracy test']:.2f}")

Bảng kiểm tra overfitting của các thuật toán:
            Thuật toán  Accuracy train  Accuracy test  Chênh lệch
3        Decision Tree        0.976510       0.812500    0.164010
2  K-Nearest Neighbors        0.828859       0.695312    0.133547
0        Random Forest        1.000000       0.890625    0.109375
4          Naive Bayes        0.865772       0.757812    0.107959
1                  SVM        1.000000       0.906250    0.093750
Thuật toán có Accuracy cao nhất: SVM với Accuracy test: 0.91
